In [19]:
pip install sklearn-genetic

Note: you may need to restart the kernel to use updated packages.


In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
import os
import glob
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import tree

In [60]:
traindata = pd.read_csv('preprocessed_train_DT_2018.csv')
testdata = pd.read_csv('preprocessed_test_DT_2018.csv')
traindata.pop("Unnamed: 0")
testdata.pop("Unnamed: 0")
traindata = traindata[20000:50000]
testdata = testdata[:20000]
traindata.head()
testdata['Label'].value_counts()

1    11383
0     8617
Name: Label, dtype: int64

In [61]:
Y = traindata.pop('Label')
X = traindata.iloc[:,0:82]
C = testdata.pop('Label')
T = testdata.iloc[:,0:82]

X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 20000 to 49999
Data columns (total 80 columns):
Dst Port             30000 non-null int64
Flow Duration        30000 non-null int64
Tot Fwd Pkts         30000 non-null int64
Tot Bwd Pkts         30000 non-null int64
TotLen Fwd Pkts      30000 non-null int64
TotLen Bwd Pkts      30000 non-null int64
Fwd Pkt Len Max      30000 non-null int64
Fwd Pkt Len Min      30000 non-null int64
Fwd Pkt Len Mean     30000 non-null float64
Fwd Pkt Len Std      30000 non-null float64
Bwd Pkt Len Max      30000 non-null int64
Bwd Pkt Len Min      30000 non-null int64
Bwd Pkt Len Mean     30000 non-null float64
Bwd Pkt Len Std      30000 non-null float64
Flow Byts/s          30000 non-null float64
Flow Pkts/s          30000 non-null float64
Flow IAT Mean        30000 non-null float64
Flow IAT Std         30000 non-null float64
Flow IAT Max         30000 non-null int64
Flow IAT Min         30000 non-null int64
Fwd IAT Tot          30000 non-

In [62]:
_traindata = np.array(X)
_trainlabel = np.array(Y)
_traindata = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
_testdata = np.array(T)
_testlabel = np.array(C)
_testdata = T.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
_trainlabel

array([0, 0, 0, ..., 1, 1, 1])

In [67]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clfSVM = make_pipeline(StandardScaler(), SVC())
clfSVM.fit(_traindata, _trainlabel)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [68]:
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score
predSVM = clfSVM.predict(_testdata)
results = confusion_matrix(_testlabel, predSVM)
error = zero_one_loss(_testlabel, predSVM)
accuracy_score = accuracy_score(_testlabel, predSVM)
print(" Results\n======================\nConfusion Matrix:\n{}\nError Value:{}\nAccuracy_Score:{}\n".format(results, error, accuracy_score))

 Results
Confusion Matrix:
[[8571   46]
 [6071 5312]]
Error Value:0.30584999999999996
Accuracy_Score:0.69415



In [17]:
from genetic_selection import GeneticSelectionCV
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


cv = GeneticSelectionCV(estimator=SVC(),
                                  cv=10,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=20,
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=40,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)

print(cv.fit(_traindata, _trainlabel))
from joblib import dump, load
dump(cv, 'svm_genetic_intrusion_detection_binary_classification_30k.pkl')

Selecting features with genetic algorithm.


In [63]:
from joblib import dump, load
clf2 = load("svm_genetic_intrusion_detection_binary_classification_30k.pkl")

In [64]:
clf2

GeneticSelectionCV(caching=True, crossover_independent_proba=0.5,
                   crossover_proba=0.5, cv=5,
                   estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   fit_params=None, max_features=5,
                   mutation_independent_proba=0.05, mutation_proba=0.2,
                   n_gen_no_change=10, n_generations=40, n_jobs=-1,
                   n_population=50, scoring='accuracy', tournament_size=3,
                   verbose=1)

In [65]:
genfeats = _traindata.columns[clf2.support_]
genfeats = list(genfeats)
print("Chosen Feats:  ", genfeats)
#Le dernier score MCC (Matthews correlation coefficient) comme score final du modèle, qui est le meilleur
cv_score = clf2.generation_scores_[-1]
print("score",cv_score) 

Chosen Feats:   ['Bwd Pkt Len Std', 'Init Fwd Win Byts']
score 0.8005666666666666


In [71]:
traindata = pd.read_csv('preprocessed_train_DT_2018.csv')
testdata = pd.read_csv('preprocessed_test_DT_2018.csv')
traindata.pop("Unnamed: 0")
testdata.pop("Unnamed: 0")

0            9
1           20
2           22
3           23
4           25
         ...  
19995    99985
19996    99987
19997    99990
19998    99993
19999    99996
Name: Unnamed: 0, Length: 20000, dtype: int64

In [72]:
Y = traindata.pop('Label')
X = traindata.iloc[:,0:82]
X = traindata[['Bwd Pkt Len Std', 'Init Fwd Win Byts']]
C = testdata.pop('Label')
T = testdata.iloc[:,0:82]
T = testdata[['Bwd Pkt Len Std', 'Init Fwd Win Byts']]

In [73]:
_traindata = np.array(X)
_trainlabel = np.array(Y)
_traindata = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
_testdata = np.array(T)
_testlabel = np.array(C)
_testdata = T.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)


In [75]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clfSVM = make_pipeline(StandardScaler(), SVC())
clfSVM.fit(_traindata, _trainlabel)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

In [76]:
from sklearn.metrics import confusion_matrix, zero_one_loss, accuracy_score
predSVM = clfSVM.predict(_testdata)
results = confusion_matrix(_testlabel, predSVM)
error = zero_one_loss(_testlabel, predSVM)
accuracy_score = accuracy_score(_testlabel, predSVM)
print(" Results\n======================\nConfusion Matrix:\n{}\nError Value:{}\nAccuracy_Score:{}\n".format(results, error, accuracy_score))

 Results
Confusion Matrix:
[[ 7996   621]
 [  591 10792]]
Error Value:0.06059999999999999
Accuracy_Score:0.9394

